<p align="center">
  <img width="200" src="https://github.com/ANG13T/fly-catcher/blob/main/assets/logo.png" alt="Fly Catcher logo" />
</p>
<h1 align="center" style="font-size:50px !important;">Fly Catcher</h1>
<p align="center">
  <i>Fly Catcher monitors for malicious ADS-B signals in the 1090MHz frequency to detect for aircraft spoofing</i>
   <br/><br/>
  <b><a href="#features-️">Learn More</a></b> | <b><a href="#build-it-yourself-️">Build Guide</a></b> | <b><a href="#detecting-for-spoofing-">Article</a></b> | <b><a href="#watch-it-in-action-">Video</a></b> | <b><a href="https://github.com/ANG13T/fly-catcher/blob/main/assets/project_report.pdf">Research Paper</a></b>
  <br/><br/>
</p>
<p align="center">

#### Created by Angelina Tsuboi

# Step #1: Parse through JSON Flight Log File

In [22]:
import json
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np

# Enter Flight Log JSON from Fly Catcher BELOW
selected_file = "./samples/testing/sample.json" 

file = open(selected_file)
json_contents = json.load(file)
aircraft = json_contents["aircraft"]

planes = []

class PlaneLog:
    def __init__(self, hex):
        self.hex = hex
        self.planes = []
    def add_plane(self, plane_record):
        self.planes.append(plane_record)

for record in aircraft:
    matched_plane = False
    for plane in planes:
        if plane.hex == record["hex"]:
            plane.add_plane(record)
            matched_plane = True
    if matched_plane == False:
        new_plane = PlaneLog(record["hex"])
        new_plane.add_plane(record)
        planes.append(new_plane)
            
file.close()

print(f"🛫 Detected {len(planes)} planes!")
for i, plane in enumerate(planes):
    print(f"Plane #{i + 1}: {plane.hex} - {len(plane.planes)} records")

def find_plane(hex):
    for plane in planes:
        if plane.hex == hex:
            return plane

🛫 Detected 20 planes!
Plane #1: 845f9f - 1 records
Plane #2: 86d624 - 1 records
Plane #3: a835c5 - 1 records
Plane #4: 4d0104 - 1 records
Plane #5: 86e4da - 1 records
Plane #6: 71c083 - 1 records
Plane #7: 76cdb9 - 1 records
Plane #8: 7c1479 - 1 records
Plane #9: 86ebb6 - 1 records
Plane #10: abba6c - 1 records
Plane #11: 7c531d - 1 records
Plane #12: ab271f - 1 records
Plane #13: 8694fa - 1 records
Plane #14: ADCF46 - 1 records
Plane #15: B49A8B - 1 records
Plane #16: B96D38 - 1 records
Plane #17: E63452 - 1 records
Plane #18: B9FE57 - 1 records
Plane #19: 8F79E5 - 1 records
Plane #20: B0BB00 - 1 records


# Step 2: Load the CNN Model

In [20]:
model = tf.keras.models.load_model("Spoof_Detection.h5")

def predict_adsb_data(adsb_message):
    try:
        feature_vector = [
            float(adsb_message.get('alt_baro', 0)),
            float(adsb_message.get('gs', 0)),
            float(adsb_message.get('track', 0)),
            float(adsb_message.get('baro_rate', 0)),
            float(adsb_message.get('lat', 0)),
            float(adsb_message.get('lon', 0)),
            float(adsb_message.get('seen_pos', 0)),
            float(adsb_message.get('messages', 0)),
            float(adsb_message.get('seen', 0)),
            float(adsb_message.get('rssi', 0)),
        ]
    except ValueError as e:
        print(f"Error processing ADS-B message: {adsb_message}, Field: {e}")
        return None

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform([feature_vector])

    prediction = model.predict(np.array(features_scaled))
    return int(round(prediction[0][0]))

# Step 3: Select an Aircraft

In [10]:
selected_aircraft = "845f9f" # Enter ICAO Designator
aircraft_data = find_plane(selected_aircraft)

if aircraft_data == None: 
    print("Aircraft not found! Please select a ICAO Designator listed in the first section.")
else:
    print(f"✈️ Plane {selected_aircraft} selected!")

✈️ Plane 845f9f selected!


# Step 4: Show Results of CNN and Details of Aircraft

In [49]:
results = predict_adsb_data(aircraft_data.planes[0])
text = ("Spoofed" if results > 0.5 else "Not Spoofed")
percentage = results * 100
print(f"{text} --- {percentage}% prediction of being spoofed")

plane_values = [{"key": "hex", "value": "Hex (24-bit ICAO identifier)"}, {"key": "type", "value": "Data Source"}, {"key": "flight", "value": "Flight Callsign"}, {"key": "r", "value": "Aircraft Registration"}, {"key": "t", "value": "Aircraft Type"}, {"key": "alt_baro", "value": "Aircraft Barometric Altitude (feet)"}, {"key": "gs", "value": "Ground Speed (knots)"}, {"key": "track", "value": "True Track (degrees)"}, {"key": "baro_rate", "value": "Rate of Change of Barometric Altitude (feet/minute)"}, {"key": "lat", "value": "Latitude"}, {"key": "lon", "value": "Longitude"}, {"key": "seen_pos", "value": "Updated Time Ago (seconds)"}, {"key": "messages", "value": "Total # of Mode S Messages"}, {"key": "seen", "value": "Last Timing Message (seconds)"}, {"key": "rssi", "value": "Signal Power (dbFs)"}]

def find_value(id):
    for x in plane_values:
        if x["key"] == id:
            return x

print("\n            Aircraft Details")
print("=========================================")
plane_data = aircraft_data.planes[0]

for i in plane_data:
    val = find_value(i)
    if val != None:
        val_data = val.get("value")
        print(f"{val_data}: {plane_data[i]}")

1/1 [==============================] - 0s 60ms/step
Not Spoofed --- 0% prediction of being spoofed

            Aircraft Details
Hex (24-bit ICAO identifier): 845f9f
Data Source: adsc
Flight Callsign: KZ51    
Aircraft Registration: JA11KZ
Aircraft Type: B748
Aircraft Barometric Altitude (feet): 31996
Ground Speed (knots): 487.0
True Track (degrees): 244.0
Rate of Change of Barometric Altitude (feet/minute): 48
Latitude: 57.32872
Longitude: -177.562752
Updated Time Ago (seconds): 873.399
Total # of Mode S Messages: 4420465
Last Timing Message (seconds): 803.5
Signal Power (dbFs): -29.2


# Step 5: Display Aircraft on Map using Folium

In [54]:
import folium
from IPython.display import display
plane_coordinates = (plane_data["lat"], plane_data["lon"])
m = folium.Map(location = plane_coordinates, tiles ='OpenStreetMap', zoom_start=3)
folium.Marker(location = plane_coordinates).add_to(m)
display(m)

# Thank you so much for trying Fly Catcher!

### To support my work, you can sponsor me on [GitHub](github.com/ANG13T)
### Check out my other socials!
- Website: [angelinatsuboi.com](angelinatsuboi.com)
- Instagram: [@angelina_tsuboi](https://instagram.com/angelina_tsuboi)
- Twitter: [@AngelinaTsuboi](https://twitter.com/AngelinaTsuboi)
- GitHub: [@ANG13T](github.com/ANG13T)
- Linkedin:[https://www.linkedin.com/in/angelina-tsuboi-322028211/](https://www.linkedin.com/in/angelina-tsuboi-322028211/)
- YouTube: [@AngelinaTsuboi](https://www.youtube.com/@AngelinaTsuboi)